In [2]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')
df.head(20)

,Unnamed: 0,file,message,subject,text,date,from,to,emotion_scores
0,240475,216050,239080,132210,6562 12510 5447 12122 14368 14125 16224 4597 1...,0,17917,39435,[0. 0.09375 0. 0.03125 0.125 0. ...
1,231468,208137,177449,121217,3833 6660 13658 446 16658 7193 6660 3958 12375...,0,14350,13150,[0.02777778 0.05555556 0.05555556 0.05555556 0...
2,231581,208236,276335,121217,10861 4628 10928 17400 7741 5489 18357 15348 1...,0,1422,13448,[0.2 0. 0. 0.2 0. 0.2 0.2 0.2 0. 0. ]
3,231582,208237,116126,121217,3306 13193 16689 2184 11333 6726 5981 7460 813...,0,1422,13447,[0.05533597 0.08498024 0.02371542 0.06719368 0...
4,231584,208239,321587,121217,6145 3023 11333 6480 3546 3776 4502 1024,0,1422,13445,[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
5,231585,208240,432763,121217,7370 15999 10732 12100 446 446 1050 11333 4502...,0,1422,13444,[0. 0. 0. 0. 0. 0. 0.5 0. 0. 0.5]
6,231589,208243,330564,121217,11096 11333 6458 8279 6018 4243 1029 11333 797...,0,1422,13443,[0.05263158 0.0877193 0.01754386 0.07017544 0...
7,231590,208244,415039,121217,6987 6044 8603 7643 7469 5470 8443 11333 5780 ...,0,1422,13442,[0. 0. 0.25 0. 0. 0.25 0. 0. 0. ...
8,231808,208440,225425,121217,3399 5112 9223 7591 5085 2478 9758 13230 2266 ...,0,12482,13678,[0. 0.11111111 0. 0.03703704 0...
9,90767,83541,467279,56716,4669 3082 3693 4755 4511 4285 2745 3329 170 29...,0,4463,31312,[0.06666667 0.06666667 0.06666667 0.06666667 0...


In [10]:
import numpy as np

emotion_scores = [
    [0., 0.09375, 0., 0.03125, 0.125, 0., 0.40625, 0., 0.03125, 0.1875],
    [0.1, 0.2, 0.6, 0.05, 0.15, 0., 0.3, 0., 0.05, 0.1],
    [0.15, 0.8, 0.1, 0.2, 0.5, 0.05, 0.25, 0., 0.1, 0.1],
    [0.2, 0.05, 0.05, 0.1, 0., 0.05, 0.2, 0.05, 0.1, 0.1],
    [0., 0.25, 0., 0.1, 0.61, 0., 0.1, 0.1, 0., 0.1],
    [0.05, 0.2, 0.05, 0.15, 0.1, 0., 0.15, 0., 0.05, 0.1],
    [0.1, 0.15, 0., 0.05, 0.1, 0.05, 0.2, 0.1, 0.4, 0.15],
    [0.3, 0.1, 0.9, 0., 0.05, 0., 0.1, 0.1, 0.15, 0.2],
    [0.05, 0.2, 0.1, 0., 0.05, 0.05, 0.1, 0.15, 0., 0.1],
    [0., 0.1, 0., 0., 0.15, 0.2, 0.1, 0., 0., 0.1],
    [0.05, 0.1, 0.15, 0.1, 0., 0.2, 0.1, 0., 0., 0.1],
    [0., 0.15, 0.05, 0.1, 0.1, 0.8, 0.05, 0., 0.1, 0.1],
    [0.1, 0.05, 0., 0.05, 0.1, 0.1, 0.15, 0., 0.05, 0.2],
    [0.15, 0.6, 0.1, 0.1, 0.05, 0.1, 0.1, 0., 0., 0.1],
    [0.7, 0.1, 0., 0.1, 0.1, 0., 0.15, 0.1, 0., 0.1],
    [0.15, 0.2, 0.1, 0., 0.05, 0.1, 0.1, 0., 0., 0.05],
    [0.1, 0.1, 0.15, 0.1, 0., 0.15, 0.1, 0., 0., 0.1],
    [0.1, 0.15, 0.1, 0.05, 0., 0.1, 0.1, 0.05, 0.05, 0.1],
    [0.05, 0.2, 0.1, 0.81, 0.1, 0., 0.1, 0.1, 0., 0.15],
    [0., 0.1, 0.15, 0., 0.1, 0.1, 0., 0.1, 0.1, 0.1],
    [0.1, 0., 0.1, 0.15, 0.1, 0., 0.1, 0., 0., 0.1],
    [0., 0.1, 0., 0.1, 0.15, 0.1, 0.1, 0., 0., 0.91],
    [0.2, 0., 0., 0.1, 0.1, 0.1, 0.1, 0., 0.81, 0.2],
    [0.1, 0.1, 0.1, 0.1, 0., 0.1, 0., 0.71, 0.1, 0.2],
    [0., 0.1, 0., 0., 0.1, 0.1, 0.65, 0., 0.1, 0.1],
    [0.05, 0., 0.1, 0.1, 0., 0.51, 0.15, 0., 0.1, 0.1],
    [0., 0.1, 0., 0.05, 0.1, 0.1, 0.15, 0., 0., 0.1],
    [0.1, 0.1, 0., 0., 0.1, 0.1, 0., 0., 0.1, 0.15],
]

vector_w = np.random.rand(10)

# Инициализация меток
for nums in emotion_scores:
    temp = np.dot(nums[:10], vector_w)  # Используем только первые 10 параметров
    nums.append(1 if temp >= 0 else 0)

# Вывод результатов
header = "| Index | " + " | ".join([f" Param{i} " for i in range(1, 11)]) + " | Label |"
print(header)
print('-' * len(header))

for i, nums in enumerate(emotion_scores):
    print(f'| {i:<5} | ' + ' | '.join([f'{num:>8.4f}' for num in nums[:-1]]) + f' | {nums[-1]:>5}|')

# Вывод вектора весов
print("\nВектор весов:")
print(f'| {"w":<6} | ' + ' | '.join([f'{w:.4f}' for w in vector_w]) + ' |')

vector_accuracy = [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0]

# Расчет ошибок
vector_errors = []
for i in range(len(emotion_scores)):
    vector_errors.append(vector_accuracy[i] - emotion_scores[i][10])  # 10 — индекс метки

print("vector_errors: ", vector_errors)

# Обновление векторов весов
while min(vector_errors) < -0.001 or max(vector_errors) > 0.001:
    delta_w = np.zeros(len(vector_w))  # Вектор дельт для обновления весов

    for i in range(len(emotion_scores)):
        delta_w += vector_errors[i] * 0.1 * np.array(emotion_scores[i][:10])  # Учитываем только 10 параметров

    # Обновление векторов весов
    vector_w += delta_w

    # Обновление меток на основе новых весов
    for nums in emotion_scores:
        temp = np.dot(nums[:10], vector_w)  # Учитываем только 10 параметров
        nums[10] = 1 if temp >= 0 else 0  # Сильно эмоциональное или слабо эмоциональное

    # Расчет ошибок
    vector_errors = []
    for i in range(len(emotion_scores)):
        vector_errors.append(vector_accuracy[i] - emotion_scores[i][10])  # 10 — индекс метки

    print("vector_errors: ", vector_errors)

# Вывод обновленных весов
print("Обновлённые веса:")
print(vector_w)


| Index |  Param1  |  Param2  |  Param3  |  Param4  |  Param5  |  Param6  |  Param7  |  Param8  |  Param9  |  Param10  | Label |
--------------------------------------------------------------------------------------------------------------------------------
| 0     |   0.0000 |   0.0938 |   0.0000 |   0.0312 |   0.1250 |   0.0000 |   0.4062 |   0.0000 |   0.0312 |   0.1875 |     1|
| 1     |   0.1000 |   0.2000 |   0.1000 |   0.0500 |   0.1500 |   0.0000 |   0.3000 |   0.0000 |   0.0500 |   0.1000 |     1|
| 2     |   0.1500 |   0.1000 |   0.1000 |   0.2000 |   0.0500 |   0.0500 |   0.2500 |   0.0000 |   0.1000 |   0.1000 |     1|
| 3     |   0.2000 |   0.0500 |   0.1500 |   0.1000 |   0.0000 |   0.0500 |   0.2000 |   0.0500 |   0.1000 |   0.1000 |     1|
| 4     |   0.0000 |   0.2500 |   0.0000 |   0.1000 |   0.1000 |   0.0000 |   0.1000 |   0.1000 |   0.0000 |   0.1000 |     1|
| 5     |   0.0500 |   0.2000 |   0.0500 |   0.1500 |   0.1000 |   0.0000 |   0.1500 |   0.0000 |   0.0500 

In [11]:
# Расчет ошибок
vector_errors = []
for i in range(len(emotion_scores)):
    vector_errors.append(vector_accuracy[i] - emotion_scores[i][10])  # 10 — индекс метки

print("vector_errors: ", vector_errors)

# Обновление векторов весов
while min(vector_errors) < -0.001 or max(vector_errors) > 0.001:
    delta_w = np.zeros(len(vector_w))  # Вектор дельт для обновления весов

    for i in range(len(emotion_scores)):
        delta_w += vector_errors[i] * 0.1 * np.array(emotion_scores[i][:10])  # Учитываем только 10 параметров

    # Обновление векторов весов
    vector_w += delta_w

    # Обновление меток на основе новых весов
    for nums in emotion_scores:
        temp = np.dot(nums[:10], vector_w)  # Учитываем только 10 параметров
        nums[10] = 1 if temp >= 0 else 0  # Сильно эмоциональное или слабо эмоциональное

    # Расчет ошибок
    vector_errors = []
    for i in range(len(emotion_scores)):
        vector_errors.append(vector_accuracy[i] - emotion_scores[i][10])  # 10 — индекс метки

    print("vector_errors: ", vector_errors)

# Вывод обновленных весов
print("Обновлённые веса:")
print(vector_w)

NameError: name 'vector_accuracy' is not defined